In [69]:
import xml.etree.ElementTree as ET
import xmltodict

path = '../../../../../bn.save/'

def read_file(file):
    with open(file, 'r') as myfile:
        xmltext=myfile.read()
    return xmltext

def get_energies_by_band(eigendict, nband):
    banddict = {}
    for i in range(nband):
        banddict[i+1] = []
        for kpoint in range(1,nkpt+1):
            banddict[i+1].append(eigendict[kpoint][i])
    return banddict

def check_if_metallic(banddict):
    metallic = False
    for i in banddict:
        if banddict[i][0] < Fermi:
            bandtype = "valence"
        else:
            bandtype = "conduction"

        for eigenvalue in banddict[i]:
            if eigenvalue < Fermi:
                current_kpt_type = "valence"
            else:
                current_kpt_type = "conduction"
            if bandtype != current_kpt_type:
                metallic = True
    return metallic

#Get dictionary with important info about HOMO/LUMO and metallicity
def get_homo_lumo_dict(eigendict, nkpt, nband, Fermi):
    ev_conv = 27.2114
    homo_bd = []
    homo_eig = []
    lumo_eig = []
    
    for kpt in range(1, nkpt+1):
        for band in range(nband):
            if eigendict[kpt][band] < Fermi:
                highest_val_bd = band +1
                highest_val_en = eigendict[kpt][band]
        homo_bd.append(highest_val_bd)
        homo_eig.append(highest_val_en)
        lumo_eig.append(eigendict[kpt][highest_val_bd])
    
    homo_energy = max(homo_eig)
    homo_kpt = homo_eig.index(max(homo_eig)) + 1
    lumo_energy = min(lumo_eig)
    lumo_kpt = lumo_eig.index(min(lumo_eig)) + 1
    band_gap = lumo_energy - homo_energy
    banddict = get_energies_by_band(eigendict,nband)
    is_metallic = check_if_metallic(banddict)
    print([x*ev_conv for x in homo_eig],[x*ev_conv for x in lumo_eig])
    
    if homo_kpt == lumo_kpt:
        gap_type = 'DIRECT'
    else:
        gap_type = 'INDIRECT'
    
    
    homo_lumo_dict = {'HOMO_ENERGY': homo_energy*ev_conv, 
                      'LUMO_ENERGY': lumo_energy*ev_conv, 
                      'BAND_GAP': band_gap*ev_conv, 
                      'HOMO_KPT_NUM': homo_kpt, 
                      'LUMO_KPT_NUM': lumo_kpt,
                      'HOMO_BAND_NUM': max(homo_bd),
                      'IS_METALLIC': is_metallic,
                      'FERMI_ENERGY': Fermi*ev_conv,
                      'GAP_TYPE': gap_type}
    return homo_lumo_dict

In [70]:
tree = ET.parse(path + 'data-file.xml')
file = path +  'data-file.xml'
xmldict = xmltodict.parse(read_file(file))

Fermi = float(xmldict['Root']['BAND_STRUCTURE_INFO']['FERMI_ENERGY']['#text'])
nband = int(xmldict['Root']['BAND_STRUCTURE_INFO']['NUMBER_OF_BANDS']['#text'])
nkpt = int(xmldict['Root']['BAND_STRUCTURE_INFO']['NUMBER_OF_K-POINTS']['#text'])

kpoint_xml_paths = []
for i in range(nkpt):
    kpoint_xml_paths.append(path + xmldict['Root']['EIGENVALUES']['K-POINT.' + str(i+1)]['DATAFILE']['@iotk_link'])
    
occdict = {}
eigendict = {}
for i, kpfile in enumerate(kpoint_xml_paths):
    occdict[i+1] = [float(q) for q in xmltodict.parse(read_file(kpfile))['Root']['OCCUPATIONS']['#text'].split('\n')]
    eigendict[i+1] = [float(q) for q in xmltodict.parse(read_file(kpfile))['Root']['EIGENVALUES']['#text'].split('\n')]

In [71]:
get_homo_lumo_dict(eigendict, nkpt, nband, Fermi)

[-3.098568399930918, -3.1509652397558416, -3.385838155308632, -3.8148516969119908, -4.061181422886841, -3.8917666016186128, -3.7260577641114025, -3.2954777511094258, -3.640748482170567, -3.942620658443245, -3.84279337651176, -3.602206348070846, -3.8838014379888035, -3.8442947064216324, -3.5961333382232783, -3.4192596347030446, -3.6191783349695417, -3.3102229040631412, -3.1202607018165485] [-0.33599197330997893, -0.4694708617333241, -0.8020859042986307, -0.927097501257557, -0.8960624491879936, -0.901218906479595, -0.8740539812129887, -0.7064355236547767, -0.9950565051555555, -1.058487522797528, -0.9315161844825524, -0.8586968005613735, -1.1628029247738192, -1.0163108036185016, -0.8962170406154063, -0.9801007557587923, -0.9329050382486463, -1.197306262500803, -1.436265582829194]


{'HOMO_ENERGY': -3.098568399930918,
 'LUMO_ENERGY': -1.436265582829194,
 'BAND_GAP': 1.662302817101724,
 'HOMO_KPT_NUM': 1,
 'LUMO_KPT_NUM': 19,
 'HOMO_BAND_NUM': 13,
 'IS_METALLIC': False,
 'FERMI_ENERGY': -2.2505657134942765,
 'GAP_TYPE': 'INDIRECT'}

In [29]:
xmltodict.parse(read_file('../../../../../bn.save/./K00001/eigenval.xml'))['Root']['EIGENVALUES']

OrderedDict([('@type', 'real'),
             ('@size', '17'),
             ('#text',
              '-2.350932570812750E+000\n-1.400700398313944E+000\n-1.393350950316323E+000\n-1.393350765205279E+000\n-6.311616158300198E-001\n-5.834078283397391E-001\n-3.194500575749127E-001\n-2.097309932530080E-001\n-2.097307940084339E-001\n-1.787524662620082E-001\n-1.689689398438817E-001\n-1.689664008106618E-001\n-1.138702308565865E-001\n-1.234747103456562E-002\n-1.234520698282502E-002\n-3.731716212568933E-003\n-3.730487330064607E-003')])

In [72]:
nkpt

19